# Seminar_1 

**Exercise 1**

In [1]:
class RGBto_YUV:
    def YUV_to_RGB(self, Y,U,V):
        B = round(1.163 * (Y -16) + 2.018 * (U - 128))
        G = round(1.164 * (Y - 16) - 0.813 * (V - 128) - 0.391 * (U - 128))
        R = round(1.164 * (Y - 16) + 1.596 * (V - 128)) 
        return(R, G, B)
    
    def RGB_to_YUV(self, R,G,B):
        Y = round(0.257 * R + 0.504 * G + 0.098 * B +16)
        U = round(-0.148 * R - 0.291 * G + 0.439 * B +128)
        V = round(0.439 * R - 0.368 * G - 0.071 * B +128)
        return(Y,U,V)
        
        

In [2]:
Y, U, V = 100, 128, 128

# Convertir YUV a RGB
converter = RGBto_YUV()
R, G, B = converter.YUV_to_RGB(Y, U, V)
print(f"Valores RGB: R = {R}, G = {G}, B = {B}")

Valores RGB: R = 98, G = 98, B = 98


In [3]:
# Convertir RGB a YUV
Y, U, V = converter.RGB_to_YUV(R, G, B)
print(f"Valores YUV: Y = {Y}, U = {U}, V = {V}")

Valores YUV: Y = 100, U = 128, V = 128


**Exercise 2**

In [4]:
import subprocess
import os


def resize_image(input_image, output_image, width, height, quality = 28):

    command = ["ffmpeg","-y","-loglevel", "info","-i", input_image,"-vf", f"scale={width}:{height}"]

    if quality is not None:
        command.extend(["-q:v", str(quality)])
    
    command.append(output_image)
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(result.stdout)
        print(f"Resized image saved to {output_image}")
    except subprocess.CalledProcessError as e:
        print("Error ocurred while running FFmpeg", e.stderr)

**Exercise 3**

We first implement it with a matrix 8x8 in order to check if the algorithm is really working. 

In [5]:
import numpy as np

matrix = np.arange(1, 65).reshape(8, 8)

def serpentine_matrix(matrix):
    result = [None] * 64  
    n = matrix.shape[0]  
    
    a = 0  
    i = 0 
    j = 0 
    result[a] = matrix[i][j]  # First value
    
    while a < 63:  
        #step 1: (one to the right)
        if j == n - 1:  
            i += 1
        else:
            j += 1

        a += 1
        if a < 64:
            result[a] = matrix[i][j]
        
        #step 2: (Diagonal down to the left until reaching j = 0)
        while i + 1 < n and j - 1 >= 0 and a < 63:
            i += 1
            j -= 1
            a += 1
            result[a] = matrix[i][j]
        
        #step 3: (one down) 
        if i == n - 1:  
            j += 1
        else:
            i += 1

        a += 1
        if a < 64:
            result[a] = matrix[i][j]
        
        #step 4: (Diagonal up to the right until reaching i = 0)
        while j + 1 < n and i - 1 >= 0 and a < 63:
            i -= 1
            j += 1
            a += 1
            result[a] = matrix[i][j]
    
    return result

# Obtener el recorrido zig-zag
zigzag_result = serpentine_matrix(matrix)

# Mostrar la matriz y el recorrido zig-zag
print("Matriz 8x8:")
print(matrix)
print("\nRecorrido Serpentine:")
print(zigzag_result)


Matriz 8x8:
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]

Recorrido Serpentine:
[1, 2, 9, 17, 10, 3, 4, 11, 18, 25, 33, 26, 19, 12, 5, 6, 13, 20, 27, 34, 41, 49, 42, 35, 28, 21, 14, 7, 8, 15, 22, 29, 36, 43, 50, 57, 58, 51, 44, 37, 30, 23, 16, 24, 31, 38, 45, 52, 59, 60, 53, 46, 39, 32, 40, 47, 54, 61, 62, 55, 48, 56, 63, 64]


Then we implemented for images. We use the resize_image function generated in the previous exercise in order to obtain a nxn image. Then we implement the zig zag function and obtain the expected output. First we implemented in Black and White, but then we realize that maybe it was needed to implemented for rgb images, so we added it to the code.

We can try and change the values of the resize image. In order to check if the algorithm is still working for this case with images, we have set the values to 16, because then we can see the complete matrix. 
Then we change it to a bigger value, 420 in this case.  

In [6]:
import numpy as np
import cv2
import subprocess

def load_image_bw(image_path):
    return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  

def load_image_rgb(image_path):
    return cv2.imread(image_path)  

def serpentine(matrix):
    n = matrix.shape[0]  
    result = [None] * (n * n)  
    
    a = 0  
    i = 0 
    j = 0 
    result[a] = matrix[i, j]  # First value
    
    while a < n * n - 1:  
        #step 1: (one to the right)
        if j == n - 1:  
            i += 1
        else:
            j += 1

        a += 1
        if a < n * n:
            result[a] = matrix[i, j]
        
        #step 2: (Diagonal down to the left until reaching j = 0)
        while i + 1 < n and j - 1 >= 0 and a < n * n - 1:
            i += 1
            j -= 1
            a += 1
            result[a] = matrix[i, j]
        
        #step 3: (one down)
        if i == n - 1:  
            j += 1
        else:
            i += 1

        a += 1
        if a < n * n:
            result[a] = matrix[i, j]
        
        #step 4: (Diagonal up to the right until reaching i = 0)
        while j + 1 < n and i - 1 >= 0 and a < n * n - 1:
            i -= 1
            j += 1
            a += 1
            result[a] = matrix[i, j]
    
    return result


# Inputs and Outputs
image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb.jpeg"
input_image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb_resize.jpeg"
resize_image(image, input_image, 420, 420, quality=28)
output_image_bw = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/serpentine_bw.jpeg"
output_image_rgb = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/serpentine_rgb.jpeg"

##################### Black and White case #####################################

#We load the resize image in BW
image_bw = load_image_bw(input_image)

if image_bw is None:
    print("Error loading the BW image")
else:
    # Print the matrix to then check it the convertion is done well. 
    print("Original Matrix in Black and White:")
    print(image_bw)


# Check if it's a nxn matrix
if image_bw is None:
    print("Error loading the BW image")
else:
    assert image_bw.shape[0] == image_bw.shape[1], "The resize BW image is not nxn."

    serpentine_result_bw = serpentine(image_bw)

    print("Serpentine of image in Black and White:")
    print(serpentine_result_bw)

    n = image_bw.shape[0]
    serpentine_image_bw = np.array(serpentine_result_bw).reshape((n, n))

    # Safe the output
    cv2.imwrite(output_image_bw, serpentine_image_bw)
    print(f"Output in BW safe in: {output_image_bw}")


############################# RGB CASE ############################
def serpentine_color(image):
    channels = cv2.split(image)  # Separate into B, G, and R channels
    serpentine_channels = []

    for idx, channel in enumerate(channels):
        serpentine_result = serpentine(channel)
        
        # Print serpentine result for each channel
        print(f"Serpentine traversal output for channel {idx}:")
        print(serpentine_result)
        
        n = channel.shape[0]
        serpentine_matrix = np.array(serpentine_result).reshape((n, n))
        serpentine_channels.append(serpentine_matrix)

    # Merge channels back into a color image
    serpentine_image_color = cv2.merge(serpentine_channels)
    return serpentine_image_color


#We load the resize image in BW
image_rgb = load_image_rgb(input_image)


if image_rgb is None:
    print("Error loading the image in rgb")
else:
    # Print the matrix to then check it the conversion is done well. 
    print("Original Matrix rgb:")
    print(image_rgb)


if image_rgb is None:
    print("Error loading the image")
else:
    # Check image is nxn
    assert image_rgb.shape[0] == image_rgb.shape[1], "Input image rgb is not nxn."
    
    serpentine_result_rbg = serpentine_color(image_rgb)
    
    print("Output matrix of rgb image",serpentine_result_rbg)
    
    cv2.imwrite(output_image_rgb, serpentine_result_rbg)
    print(f"Serpentine image in rgb safe in: {output_image_rgb}")



Resized image saved to /Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb_resize.jpeg
Original Matrix in Black and White:
[[216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]
 ...
 [216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]]
Serpentine of image in Black and White:
[216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 21

**Exercise 4**
We transform the rgb output to Black and White. 

In [7]:
def compress_to_bw(input_image_path, output_image_path):
    try:
        subprocess.run([
            'ffmpeg', '-i', input_image_path, 
            '-vf', 'format=gray',  # Convert to black and white
            '-compression_level', '10',  # Max compression for PNG (range 0-10)
            '-qscale:v', '31',  # Highest compression for JPEG (range 2-31)
            '-y', output_image_path
        ], check=True)
        
        print(f"Compressed and converted image saved at: {output_image_path}")
        
    except subprocess.CalledProcessError as e:
        print(f"FFMPEG failed with error: {e}")

**Exercise 5:**


In [8]:
def encoding(byte_sequence):
    if not isinstance(byte_sequence, (bytes, bytearray)):
        raise ValueError("Input must be a bytes or bytearray object.")
    
    encoded_bytes = bytearray()
    i = 0
    
    while i < len(byte_sequence):
        count = 1
        current_byte = byte_sequence[i]
        j = i

        while j < len(byte_sequence) - 1:
            if byte_sequence[j] == byte_sequence[j + 1]:
                count += 1
                j += 1
            else:
                break

        encoded_bytes.append(count)
        encoded_bytes.append(current_byte)
        
        i = j + 1
    
    return bytes(encoded_bytes)

#example
input_bytes = bytes([0, 65, 65, 65, 66, 66, 66, 67, 67, 68, 65, 65])
encoded_result = encoding(input_bytes)
print(list(encoded_result)) 

[1, 0, 3, 65, 3, 66, 2, 67, 1, 68, 2, 65]


**Exercise 6**

In [9]:
import numpy as np
from scipy.fftpack import dct, idct


class DCT:
    
    def encode_dct(self, input):    
        return dct(input, type=2, norm='ortho')
    
    def decode_dct(self, input_inverse):
        return idct(input_inverse, type=2, norm='ortho')

# example:
if __name__ == "__main__":
    dct_processor = DCT()

    input = np.array([
        [52, 55, 61, 66, 70, 61, 64, 73],
        [63, 59, 66, 90, 109, 85, 69, 72],
        [62, 59, 68, 113, 144, 104, 66, 73],
        [63, 58, 71, 122, 154, 106, 70, 69],
    ])

    # DCT
    dct_encode = dct_processor.encode_dct(input)
    print("DCT Result:\n", dct_encode)

    # IDCT 
    reconstructed_block = dct_processor.decode_dct(dct_encode)
    print("\nReconstructed Block:\n", reconstructed_block)

DCT Result:
 [[177.48380208 -14.43003934  -5.65536258  -6.74138401   7.07106781
   -3.08289291  -0.71893958   2.41318823]
 [216.72822843 -15.7021571  -33.96500451  11.82921573  19.44543648
   -7.34845902  -1.62125521   3.31844356]
 [243.59828612 -21.32862687 -65.34971214  22.37520692  33.58757211
  -16.02229259  -1.63252036   1.10721787]
 [252.08356749 -20.77509775 -75.89507043  24.72899923  36.41599923
  -12.49959345  -4.91815858   3.8899967 ]]

Reconstructed Block:
 [[ 52.  55.  61.  66.  70.  61.  64.  73.]
 [ 63.  59.  66.  90. 109.  85.  69.  72.]
 [ 62.  59.  68. 113. 144. 104.  66.  73.]
 [ 63.  58.  71. 122. 154. 106.  70.  69.]]


In [ ]:
#pip install PyWavelets

   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.3 MB 8.5 MB/s eta 0:00:01
   ------------------- -------------------- 2.1/4.3 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------  4.2/4.3 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


**Exercise 7**

In [15]:
import pywt
import numpy as np

class DWT:
    def __init__(self, wavelet='db2', mode='smooth'):
        self.wavelet = wavelet  # db1: simplest wavelet. db2:used for more complex transformations
        self.mode = mode        # boundary handling. smooth: smooth continuity at the boundaries. periodic: periodic data

    def encode_dwt(self, input_signal):
        
        cA, cD = pywt.dwt(input_signal, self.wavelet, self.mode)
        return cA, cD
    
    def decode_dwt(self, cA, cD):

        return pywt.idwt(cA, cD, self.wavelet, self.mode)

# example
if __name__ == "__main__":

    wavelet_processor = DWT(wavelet='db2', mode='smooth')
    
    input_signal = [1, 2, 3, 4, 5, 6]

    cA, cD = wavelet_processor.encode_dwt(input_signal)
    print("Approximation coefficients (cA):", cA)
    print("Detail coefficients (cD):", cD)
    
    reconstructed_signal = wavelet_processor.decode_dwt(cA, cD)
    print("\nReconstructed Signal:", reconstructed_signal)


Approximation coefficients (cA): [-0.51763809  2.31078903  5.13921616  7.96764328]
Detail coefficients (cD): [0.00000000e+00 1.66533454e-16 3.33066907e-16 2.22044605e-16]

Reconstructed Signal: [1. 2. 3. 4. 5. 6.]


**Exercise 8**

In [16]:
import unittest
from unittest.mock import patch, MagicMock

class TestRGBtoYUV(unittest.TestCase):
    def test_YUV_to_RGB(self):
        yuv_converter = RGBto_YUV()
        R, G, B = yuv_converter.YUV_to_RGB(16, 128, 128)
        self.assertEqual((R, G, B), (0, 0, 0), "YUV to RGB conversion for (16,128,128) should be (0,0,0)")

    def test_RGB_to_YUV(self):
        yuv_converter = RGBto_YUV()
        Y, U, V = yuv_converter.RGB_to_YUV(0, 0, 0)
        self.assertEqual((Y, U, V), (16, 128, 128), "RGB to YUV conversion for (0,0,0) should be (16,128,128)")

class TestResizeImage(unittest.TestCase):
    def test_resize_image_success(self):
        input_image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb_1.jpg"
        output_image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/resized_image.jpg"
        width = 320
        height = 240
        quality = 28

        # Llamar a la función sin "mock"
        resize_image(input_image, output_image, width, height, quality)

        # Verificar si el archivo se creó
        import os
        self.assertTrue(os.path.exists(output_image), "El archivo de salida no se creó correctamente")
        
        
class TestSerpentine(unittest.TestCase):

    def setUp(self):
        # Input image       
        self.image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb.jpeg"
        self.input_image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb_resize_serpentine.jpeg"
        resize_image(image, input_image, 8, 8, quality=28) #We resize it, to obtain a nxn matrix. 

    def test_serpentine_bw(self):
    
        image_bw = cv2.imread(self.input_image, cv2.IMREAD_GRAYSCALE)
        
        # Check if we have load correctly the image
        self.assertIsNotNone(image_bw, "Failed to load the image in BW")

        # Check if the image is (nxn)
        self.assertEqual(image_bw.shape[0], image_bw.shape[1], "The BW image is not nxn")
        
        # Print matrix image
        print("Original Matrix in BW:")
        print(image_bw)

        # We apply the function
        serpentine_result = serpentine(image_bw)
        
        # print of the result
        print("Matrix after serpentine in BW:")
        print(serpentine_result)

        # Check if the result is correct. 
        self.assertEqual(len(serpentine_result), image_bw.size, "Serpentine result has incorrect size")

    def test_serpentine_color(self):
        
        image_rgb = cv2.imread(self.input_image)

        # Check if we have load correctly the image
        self.assertIsNotNone(image_rgb, "Failed to load the RGB image")

        # Check if the image is (nxn)
        self.assertEqual(image_rgb.shape[0], image_rgb.shape[1], "The RGB image is not nxn")
        
        # Print matrix image
        print("Original Matrix in RGB:")
        print(image_rgb)

        # We apply the function
        serpentine_result = serpentine_color(image_rgb)
        
        # print of the result
        print("Matrix after serpentine in RGB:")
        print(serpentine_result)

        # Check if the result is correct. 
        self.assertEqual(serpentine_result.shape, image_rgb.shape, "The serpentine color result has incorrect size")

        # Check if the 3 channels are correctly transformed
        red_channel, green_channel, blue_channel = cv2.split(serpentine_result)

        # Check if each channel has the same size as the original image
        self.assertEqual(red_channel.shape, image_rgb.shape[:2], "Red channel size mismatch")
        self.assertEqual(green_channel.shape, image_rgb.shape[:2], "Green channel size mismatch")
        self.assertEqual(blue_channel.shape, image_rgb.shape[:2], "Blue channel size mismatch")

class TestCompressToBW(unittest.TestCase):

    def test_compress_to_bw_success(self):
        # Define the paths (you can use actual image paths for testing)
        input_image_path = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb.jpeg"
        output_image_path =  "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/output_image_compressed.jpeg"

        # Ensure the output path doesn't exist before running the test
        if os.path.exists(output_image_path):
            os.remove(output_image_path)

        # Call the function
        compress_to_bw(input_image_path, output_image_path)
        
        # Check if the output image was created
        self.assertTrue(os.path.exists(output_image_path), "Output image was not created")


# Ejecuta las pruebas
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....

Compressed and converted image saved at: /Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/output_image_compressed.jpeg

Resized image saved to /Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/resized_image.jpg

Resized image saved to /Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb_resize.jpeg
Original Matrix in BW:
[[231 224 148 141 141 148 224 231]
 [209 208 141 140 140 141 208 209]
 [211 210 143 142 142 143 210 211]
 [238 225 141 129 129 141 225 238]
 [246 220 117  90  90 117 220 246]
 [225 200  99  74  74  99 200 225]
 [212 206 132 126 126 132 206 212]
 [219 232 185 199 199 185 232 219]]
Matrix after serpentine in BW:
[231, 224, 209, 211, 208, 148, 141, 141, 210, 238, 246, 225, 143, 140, 141, 148, 140, 142, 141, 220, 225, 212, 200, 117, 129, 142, 141, 224, 231, 208, 143, 129, 90, 99, 206, 219, 232, 132, 74, 90, 141, 210, 209, 211, 225, 117, 74, 126, 185, 199, 126, 99, 220, 238, 246, 200, 132, 199, 185, 206, 225, 2

.


Resized image saved to /Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/seminar1/logo_fcb_resize.jpeg
Original Matrix in RGB:
[[[ 20 252 255]
  [ 40 240 255]
  [ 17 153 187]
  [ 29 143 180]
  [ 17 145 180]
  [  0 158 187]
  [ 33 241 255]
  [ 18 252 255]]

 [[  0 230 248]
  [ 24 224 247]
  [ 10 146 180]
  [ 28 142 179]
  [ 16 144 179]
  [  0 151 180]
  [ 17 225 247]
  [  0 230 248]]

 [[  0 232 250]
  [ 26 226 249]
  [ 12 148 182]
  [ 30 144 181]
  [ 18 146 181]
  [  0 153 182]
  [ 19 227 249]
  [  0 232 250]]

 [[ 27 255 255]
  [ 41 241 255]
  [ 10 146 180]
  [ 17 131 168]
  [  5 133 168]
  [  0 151 180]
  [ 34 242 255]
  [ 25 255 255]]

 [[ 35 255 255]
  [ 36 236 255]
  [  0 122 156]
  [  0  92 129]
  [  0  94 129]
  [  0 127 156]
  [ 29 237 255]
  [ 33 255 255]]

 [[ 14 246 255]
  [ 16 216 239]
  [  0 104 138]
  [  0  76 113]
  [  0  78 113]
  [  0 109 138]
  [  9 217 239]
  [ 12 246 255]]

 [[  1 233 251]
  [ 22 222 245]
  [  1 137 171]
  [ 14 128 165]
  [  2 130 165]
  


----------------------------------------------------------------------
Ran 6 tests in 0.448s

OK


In [10]:
class TestEncoding(unittest.TestCase):
    def test_encoding(self):
        input_bytes = bytes([0, 65, 65, 65, 66, 66, 66, 67, 67, 68, 65, 65])
        encoded = encoding(input_bytes)
        self.assertEqual(list(encoded), [1, 0, 3, 65, 3, 66, 2, 67, 1, 68, 2, 65])

    def test_encoding_non_bytes_input(self):
        with self.assertRaises(ValueError):
            encoding([1, 2, 3])  # Input that is not a byte sequence

class TestDCT(unittest.TestCase):
    def setUp(self):
        self.dct_processor = DCT()
        self.input_block = np.array([
            [52, 55, 61, 66, 70, 61, 64, 73],
            [63, 59, 66, 90, 109, 85, 69, 72],
            [62, 59, 68, 113, 144, 104, 66, 73],
            [63, 58, 71, 122, 154, 106, 70, 69],
            [67, 61, 68, 104, 126, 88, 68, 70],
            [79, 65, 60, 70, 77, 68, 58, 75],
            [85, 71, 64, 59, 55, 61, 65, 83],
            [87, 79, 69, 68, 65, 76, 78, 94]
        ])

    def test_encode_dct(self):
        dct_result = self.dct_processor.encode_dct(self.input_block)
        self.assertIsNotNone(dct_result)  # Ensure result is not None

    def test_decode_dct(self):
        dct_result = self.dct_processor.encode_dct(self.input_block)
        reconstructed = self.dct_processor.decode_dct(dct_result)
        np.testing.assert_array_almost_equal(self.input_block, reconstructed, decimal=0)

class TestDWT(unittest.TestCase):
    def setUp(self):
        self.wavelet_processor = DWT(wavelet='db2', mode='smooth')
        self.input_signal = [1, 2, 3, 4, 5, 6]

    def test_encode_dwt(self):
        cA, cD = self.wavelet_processor.encode_dwt(self.input_signal)
        self.assertIsNotNone(cA)
        self.assertIsNotNone(cD)

    def test_decode_dwt(self):
        cA, cD = self.wavelet_processor.encode_dwt(self.input_signal)
        reconstructed_signal = self.wavelet_processor.decode_dwt(cA, cD)
        np.testing.assert_array_almost_equal(self.input_signal, reconstructed_signal, decimal=1)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.............
----------------------------------------------------------------------
Ran 13 tests in 0.013s

OK
